In [3]:
%matplotlib inline 
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report


#K-Means import
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import LatentDirichletAllocation

In [4]:
def bench_k_means(estimator, name, data,labels=None):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=None)))
    return estimator

# K-Means Fold 1

In [5]:
from sklearn.dummy import DummyClassifier
traindata=pd.read_csv("../data/tsv/test1.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train1.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2)))])
clf.fit(X_train)

X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

lda = LatentDirichletAllocation(n_components=4,random_state=42)
lda.fit(X_train)

data=lda.transform(X_train)
labels=np.array(Y_train)-1
# print(labels)
X_test=lda.transform(X_test)
# Do K-Means
for rstate in [27]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))

Random State 27
__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.02s	0	0.028	0.027	0.028	0.039	0.014	0.976
random   	0.02s	0	0.028	0.027	0.028	0.039	0.014	0.976
PCA-based	0.00s	0	0.028	0.027	0.028	0.039	0.014	0.976
__________________________________________________________________________________
kmeans++ Initialization
              precision    recall  f1-score   support

           0    0.33333   0.10345   0.15789        58
           1    0.47586   0.76667   0.58723        90
           2    0.00000   0.00000   0.00000        12
           3    0.28571   0.16667   0.21053        24

    accuracy                        0.42935       184
   macro avg    0.27373   0.25920   0.23891       184
weighted avg    0.37510   0.42935   0.36447       184

Random Initialization
              precision    recall  f1-score   support

           0    0.33333   0.10345   0.15789        58
          

# K-MeansFold 2

In [6]:
from sklearn.dummy import DummyClassifier
traindata=pd.read_csv("../data/tsv/test2.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train2.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2)))])
clf.fit(X_train)

X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

lda = LatentDirichletAllocation(n_components=4,random_state=42)
lda.fit(X_train)

data=lda.transform(X_train)
labels=np.array(Y_train)-1
# print(labels)
X_test=lda.transform(X_test)
# print(labels)

# Do K-Means
# Do K-Means
for rstate in [27]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test)-1,ypred,digits=5))


Random State 27
__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.03s	1	0.023	0.020	0.021	0.007	0.008	0.969
random   	0.03s	1	0.023	0.020	0.021	0.007	0.008	0.969
PCA-based	0.00s	1	0.023	0.020	0.021	0.007	0.008	0.969
__________________________________________________________________________________
kmeans++ Initialization
              precision    recall  f1-score   support

           0    0.09091   0.03571   0.05128        56
           1    0.43939   0.32222   0.37179        90
           2    0.09091   0.40000   0.14815        15
           3    0.03333   0.04348   0.03774        23

    accuracy                        0.20652       184
   macro avg    0.16364   0.20035   0.15224       184
weighted avg    0.25417   0.20652   0.21426       184

Random Initialization
              precision    recall  f1-score   support

           0    0.39394   0.46429   0.42623        56
          

# Testing Autoclustering with Elbow Method


In [ ]:
# FOld -1

from sklearn.dummy import DummyClassifier
traindata=pd.read_csv("../data/tsv/test1.tsv",delimiter="\t")
X_test=traindata.text.values
Y_test=np.array(traindata.label.values).astype(np.int32)

traindata=pd.read_csv("../data/tsv/train1.tsv",delimiter="\t")
X_train=traindata.text.values
Y_train=np.array(traindata.label.values).astype(np.int32)
clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('tfidf', TfidfTransformer(use_idf=True))])
clf.fit(X_train)


X_train=(clf.transform(X_train)).toarray()
X_test=clf.transform(X_test)

data=X_train
labels=np.array(Y_train)-1
# print(labels)


distortions = []
for i in range(1, 100):
    km = KMeans(init='k-means++', n_clusters=i, n_init=10,random_state=rstate)
    km.fit(data)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 100), distortions, marker='-')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

# K-Means with Learnt BERT Representations

In [56]:
import pickle

modelfolder="../data/features/bert/fold1/"

X_train= pickle.load(open(modelfolder+'xtrain.pkl', 'rb'))
Y_train= pickle.load(open(modelfolder+'ytrain.pkl', 'rb'))
X_test= pickle.load(open(modelfolder+'xtest.pkl', 'rb'))
Y_test= pickle.load(open(modelfolder+'ytest.pkl', 'rb'))



data=X_train
labels=np.array(Y_train)
# print(labels)

# Do K-Means
for rstate in [23]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

Random State 23
__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.28s	3385	0.027	0.027	0.027	0.028	0.013	0.226
random   	0.31s	3387	0.023	0.023	0.023	0.011	0.009	0.205
PCA-based	0.03s	3387	0.024	0.024	0.024	0.026	0.010	0.234
__________________________________________________________________________________
kmeans++ Initialization
              precision    recall  f1-score   support

           0    0.47368   0.31034   0.37500        58
           1    0.67273   0.41111   0.51034        90
           2    0.00000   0.00000   0.00000        12
           3    0.16667   0.62500   0.26316        24

    accuracy                        0.38043       184
   macro avg    0.32827   0.33661   0.28713       184
weighted avg    0.50010   0.38043   0.40216       184

Random Initialization
              precision    recall  f1-score   support

           0    0.23636   0.22414   0.23009        58
 

In [59]:
import pickle

modelfolder="../data/features/bert/fold2/"

X_train= pickle.load(open(modelfolder+'xtrain.pkl', 'rb'))
Y_train= pickle.load(open(modelfolder+'ytrain.pkl', 'rb'))
X_test= pickle.load(open(modelfolder+'xtest.pkl', 'rb'))
Y_test= pickle.load(open(modelfolder+'ytest.pkl', 'rb'))

data=X_train
labels=np.array(Y_train)
# print(labels)

# Do K-Means
for rstate in [1]:
    
    print("Random State", rstate)
    
    print(82 * '_')
    print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
    km1_km=bench_k_means(KMeans(init='k-means++', n_clusters=4, n_init=10,random_state=rstate), name="k-means++", data=data,labels=labels)

    km1_random=bench_k_means(KMeans(init='random', n_clusters=4, n_init=10,random_state=rstate), name="random", data=data,labels=labels)


    pca = PCA(n_components=4).fit(data) 
    km1_pca=bench_k_means(KMeans(init=pca.components_, n_clusters=4, n_init=1,random_state=rstate), name="PCA-based", data=data,labels=labels)
    print(82 * '_')


    print("kmeans++ Initialization")
    ypred=km1_km.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

    print("Random Initialization")
    ypred=km1_random.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))


    print("PCA Components Initialization")
    ypred=km1_pca.predict(X_test)
    print(classification_report(np.array(Y_test),ypred,digits=5))

Random State 1
__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.35s	3281	0.023	0.024	0.023	0.030	0.008	0.247
random   	0.37s	3293	0.034	0.035	0.035	0.035	0.020	0.232
PCA-based	0.05s	3293	0.022	0.022	0.022	0.023	0.008	0.241
__________________________________________________________________________________
kmeans++ Initialization
              precision    recall  f1-score   support

           0    0.29630   0.42857   0.35036        56
           1    0.50000   0.38889   0.43750        90
           2    0.16667   0.33333   0.22222        15
           3    0.33333   0.04348   0.07692        23

    accuracy                        0.35326       184
   macro avg    0.32407   0.29857   0.27175       184
weighted avg    0.39000   0.35326   0.34836       184

Random Initialization
              precision    recall  f1-score   support

           0    0.35897   0.25000   0.29474        56
  